In [1]:
import pandas as pd

from datetime import datetime
import glob, os
import time

In [ ]:
FILES_FOLDER = os.path.dirname(os.path.abspath('FZ-Data'))

In [ ]:
def read_file(idx, file_path):
    df = pd.read_excel(file_path, sheet_name='smartMeter')
    
    if set(['Timestamp','total_energy_in', 'total_energy_out']).issubset(df.columns):
        df = df[['Timestamp', 'total_energy_in', 'total_energy_out']]

        df['date'] = [datetime.fromtimestamp(date) for date in df['Timestamp']]
        df.drop(columns=['Timestamp'], inplace=True)
        df.set_index('date', inplace=True)
        df.columns = [str(idx+1) + '_cons', str(idx+1) + '_deli']
        df.index = df.index.round(freq = 'T')
        print('File ' + file + ' read')
    else:
        print('FILE ' + file + ' CORRUPTED!')
    
    return df

## Creating new DataSets (20 houses per dataset)

In [ ]:
os.chdir(FILES_FOLDER + '/FZ-Data')

files_list = glob.glob("*.xlsx")
files_list.sort()

df_full = pd.DataFrame()
for idx, file in enumerate(files_list):
    start = time.time()
    
    df_to_insert = read_file(idx, file)
    df_full = df_full.join(df_to_insert, how='outer')
    
    time_e = time.time() - start
    print('time: ' + str(time_e))
    
    if (idx+1) % 20 == 0:
        df_full.to_csv('df_full_' + str(idx) +'.csv')
        del df_full
        df_full = pd.DataFrame()